In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import transforms
import math
from PIL import Image
from torchsummary import summary

%matplotlib inline

h, w = 64, 64

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cuda:0


In [2]:
def resize (img):
    im = Image.fromarray(img)
    im.thumbnail((h, w), Image.ANTIALIAS)
    im = np.array(im).astype('float32')
    return np.pad(im, (((h - im.shape[0]) // 2, (h - im.shape[0] + 1) // 2), ((w - im.shape[1]) // 2, (w - im.shape[1] + 1) // 2)), 'constant', constant_values=255)

In [3]:
data1 = np.load('train-1.npy')
data2 = np.load('train-2.npy')
data3 = np.load('train-3.npy')
data4 = np.load('train-4.npy')
train = np.concatenate((data1, data2, data3, data4), axis=0)

train[:, 0] = [resize(img) for img in tqdm(train[:, 0])]

del data1
del data2
del data3 
del data4

  3%|▎         | 9989/332987 [00:02<01:12, 4427.06it/s]

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(16, 20))
for n, (image, tag) in enumerate(train, 1):
    if n > 64:
        break
    plt.subplot(8, 8, n)
    plt.title(tag)
    plt.imshow(image, cmap='gray')

In [ ]:
print(train[10, 0].shape)

In [4]:
dct = dict(zip(range(1000), np.unique(train[:, 1])))
dct = {v:k for k,v in dct.items()}

train[:, 1] = [dct[code] for code in train[:, 1]]

#train, test = train_test_split(train, test_size=0.2, random_state=113)

In [5]:
train_x_np, train_y_np = np.array([i[:, :, np.newaxis] for i in tqdm(train[:, 0])]), np.array(train[:, 1], dtype='int32')
del train
#test_x_np, test_y_np = np.array([i[:, :, np.newaxis] for i in tqdm(test[:, 0])]), np.array(test[:, 1], dtype='int32')
#del test

100%|██████████| 332987/332987 [00:00<00:00, 1383371.11it/s]


In [6]:
import gc
gc.collect()

60

In [7]:
class Hieroglyph_data(Dataset):
    def __init__(self, tX, tY = None,
                 transform = transforms.Compose(
                     [transforms.ToPILImage(), 
                      transforms.ToTensor(), 
                      transforms.Normalize(mean=(0.5,), std=(0.5,))]), train=True):
        
        if train == True:
            self.X = tX
            self.y = torch.Tensor(tY).type(torch.LongTensor)
            self.transform = transform
            self.train = True
            
        else:
            self.X = tX
            self.y = None
            self.transform = transform
            self.train = False
            
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.train == True:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

In [8]:
train_dataset = Hieroglyph_data(train_x_np, train_y_np, transform = transforms.Compose(
                                [transforms.ToPILImage(), 
                                 transforms.RandomAffine(
                                 degrees=(-10, 10), 
                                 translate=(.1, .1), 
                                 scale=(.9, 1.1),
                                 shear=(-10, 10)),
                                 transforms.ToTensor(), 
                                 transforms.Normalize(mean=(0.5,), std=(0.5,))]))

#test_dataset = Hieroglyph_data(test_x_np, test_y_np)

In [9]:
num_classes = 1000
batch_size = 64
num_epochs = 10

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, pin_memory = True)
#test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, pin_memory = True)

In [10]:
'''Dual Path Networks in PyTorch.'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    def __init__(self, last_planes, in_planes, out_planes, dense_depth, stride, first_layer):
        super(Bottleneck, self).__init__()
        self.out_planes = out_planes
        self.dense_depth = dense_depth

        self.conv1 = nn.Conv2d(last_planes, in_planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=32, bias=False)
        self.bn2 = nn.BatchNorm2d(in_planes)
        self.conv3 = nn.Conv2d(in_planes, out_planes+dense_depth, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes+dense_depth)

        self.shortcut = nn.Sequential()
        if first_layer:
            self.shortcut = nn.Sequential(
                nn.Conv2d(last_planes, out_planes+dense_depth, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_planes+dense_depth)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        x = self.shortcut(x)
        d = self.out_planes
        out = torch.cat([x[:,:d,:,:]+out[:,:d,:,:], x[:,d:,:,:], out[:,d:,:,:]], 1)
        out = F.relu(out)
        return out


class DPN(nn.Module):
    def __init__(self, cfg):
        super(DPN, self).__init__()
        in_planes, out_planes = cfg['in_planes'], cfg['out_planes']
        num_blocks, dense_depth = cfg['num_blocks'], cfg['dense_depth']

        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.last_planes = 64
        self.layer1 = self._make_layer(in_planes[0], out_planes[0], num_blocks[0], dense_depth[0], stride=1)
        self.layer2 = self._make_layer(in_planes[1], out_planes[1], num_blocks[1], dense_depth[1], stride=2)
        self.layer3 = self._make_layer(in_planes[2], out_planes[2], num_blocks[2], dense_depth[2], stride=2)
        self.layer4 = self._make_layer(in_planes[3], out_planes[3], num_blocks[3], dense_depth[3], stride=2)
        self.linear = nn.Linear(out_planes[3]+(num_blocks[3]+1)*dense_depth[3], 1000)

    def _make_layer(self, in_planes, out_planes, num_blocks, dense_depth, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for i,stride in enumerate(strides):
            layers.append(Bottleneck(self.last_planes, in_planes, out_planes, dense_depth, stride, i==0))
            self.last_planes = out_planes + (i+2) * dense_depth
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def DPN26():
    cfg = {
        'in_planes': (96,192,384,768),
        'out_planes': (256,512,1024,2048),
        'num_blocks': (2,2,2,2),
        'dense_depth': (16,32,24,128)
    }
    return DPN(cfg)

def DPN92():
    cfg = {
        'in_planes': (96,192,384,768),
        'out_planes': (256,512,1024,2048),
        'num_blocks': (3,4,20,3),
        'dense_depth': (16,32,24,128)
    }
    return DPN(cfg)

model = DPN92()

model = model.to(device)

summary(model, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,600
       BatchNorm2d-2           [-1, 64, 30, 30]             128
            Conv2d-3           [-1, 96, 30, 30]           6,144
       BatchNorm2d-4           [-1, 96, 30, 30]             192
            Conv2d-5           [-1, 96, 30, 30]           2,592
       BatchNorm2d-6           [-1, 96, 30, 30]             192
            Conv2d-7          [-1, 272, 30, 30]          26,112
       BatchNorm2d-8          [-1, 272, 30, 30]             544
            Conv2d-9          [-1, 272, 30, 30]          17,408
      BatchNorm2d-10          [-1, 272, 30, 30]             544
       Bottleneck-11          [-1, 288, 30, 30]               0
           Conv2d-12           [-1, 96, 30, 30]          27,648
      BatchNorm2d-13           [-1, 96, 30, 30]             192
           Conv2d-14           [-1, 96,

In [11]:
error = nn.CrossEntropyLoss()
#error = error.to(device)

learning_rate = 0.002

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

In [12]:
print('wtf')

def train(epoch):
    model.train()

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.view(-1, 1, h, w).to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = error(outputs, labels)
        
        loss.backward()
            
        optimizer.step()
        
        if (i + 1)% 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (i+ 1) * len(images), len(train_loader.dataset),
                100. * (i + 1) / len(train_loader), loss.item()))
            
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(data_loader):
            images, labels = images.view(-1, 1, h, w).to(device), labels.to(device)

            outputs = model(images)

            loss += F.cross_entropy(outputs, labels, size_average=False).item()

            pred = outputs.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

wtf


In [13]:
for epoch in range(num_epochs):
    lr_scheduler.step()
    train(epoch)
#    evaluate(test_loader)

Train Epoch: 0 [3200/332987 (1%)]	Loss: 6.912961
Train Epoch: 0 [6400/332987 (2%)]	Loss: 6.902548
Train Epoch: 0 [9600/332987 (3%)]	Loss: 6.906547
Train Epoch: 0 [12800/332987 (4%)]	Loss: 6.908698
Train Epoch: 0 [16000/332987 (5%)]	Loss: 6.922197
Train Epoch: 0 [19200/332987 (6%)]	Loss: 6.912653
Train Epoch: 0 [22400/332987 (7%)]	Loss: 6.902184
Train Epoch: 0 [25600/332987 (8%)]	Loss: 6.911507
Train Epoch: 0 [28800/332987 (9%)]	Loss: 6.888675
Train Epoch: 0 [32000/332987 (10%)]	Loss: 6.900495
Train Epoch: 0 [35200/332987 (11%)]	Loss: 6.876614
Train Epoch: 0 [38400/332987 (12%)]	Loss: 6.571973
Train Epoch: 0 [41600/332987 (12%)]	Loss: 6.481762
Train Epoch: 0 [44800/332987 (13%)]	Loss: 6.357326
Train Epoch: 0 [48000/332987 (14%)]	Loss: 6.484236
Train Epoch: 0 [51200/332987 (15%)]	Loss: 6.329052
Train Epoch: 0 [54400/332987 (16%)]	Loss: 6.283402
Train Epoch: 0 [57600/332987 (17%)]	Loss: 6.130381
Train Epoch: 0 [60800/332987 (18%)]	Loss: 6.093709
Train Epoch: 0 [64000/332987 (19%)]	Loss: 6

Train Epoch: 1 [182400/332987 (55%)]	Loss: 0.183904
Train Epoch: 1 [185600/332987 (56%)]	Loss: 0.133254
Train Epoch: 1 [188800/332987 (57%)]	Loss: 0.061122
Train Epoch: 1 [192000/332987 (58%)]	Loss: 0.254653
Train Epoch: 1 [195200/332987 (59%)]	Loss: 0.187228
Train Epoch: 1 [198400/332987 (60%)]	Loss: 0.260102
Train Epoch: 1 [201600/332987 (61%)]	Loss: 0.305662
Train Epoch: 1 [204800/332987 (62%)]	Loss: 0.269894
Train Epoch: 1 [208000/332987 (62%)]	Loss: 0.132259
Train Epoch: 1 [211200/332987 (63%)]	Loss: 0.209872
Train Epoch: 1 [214400/332987 (64%)]	Loss: 0.427169
Train Epoch: 1 [217600/332987 (65%)]	Loss: 0.206499
Train Epoch: 1 [220800/332987 (66%)]	Loss: 0.193435
Train Epoch: 1 [224000/332987 (67%)]	Loss: 0.136257
Train Epoch: 1 [227200/332987 (68%)]	Loss: 0.405400
Train Epoch: 1 [230400/332987 (69%)]	Loss: 0.161874
Train Epoch: 1 [233600/332987 (70%)]	Loss: 0.179220
Train Epoch: 1 [236800/332987 (71%)]	Loss: 0.185250
Train Epoch: 1 [240000/332987 (72%)]	Loss: 0.410193
Train Epoch:

Train Epoch: 3 [25600/332987 (8%)]	Loss: 0.026154
Train Epoch: 3 [28800/332987 (9%)]	Loss: 0.006249
Train Epoch: 3 [32000/332987 (10%)]	Loss: 0.068972
Train Epoch: 3 [35200/332987 (11%)]	Loss: 0.085608
Train Epoch: 3 [38400/332987 (12%)]	Loss: 0.056869
Train Epoch: 3 [41600/332987 (12%)]	Loss: 0.027435
Train Epoch: 3 [44800/332987 (13%)]	Loss: 0.078458
Train Epoch: 3 [48000/332987 (14%)]	Loss: 0.034140
Train Epoch: 3 [51200/332987 (15%)]	Loss: 0.052429
Train Epoch: 3 [54400/332987 (16%)]	Loss: 0.011962
Train Epoch: 3 [57600/332987 (17%)]	Loss: 0.048658
Train Epoch: 3 [60800/332987 (18%)]	Loss: 0.048390
Train Epoch: 3 [64000/332987 (19%)]	Loss: 0.068798
Train Epoch: 3 [67200/332987 (20%)]	Loss: 0.069478
Train Epoch: 3 [70400/332987 (21%)]	Loss: 0.100150
Train Epoch: 3 [73600/332987 (22%)]	Loss: 0.131322
Train Epoch: 3 [76800/332987 (23%)]	Loss: 0.110854
Train Epoch: 3 [80000/332987 (24%)]	Loss: 0.014543
Train Epoch: 3 [83200/332987 (25%)]	Loss: 0.051788
Train Epoch: 3 [86400/332987 (26%

Train Epoch: 4 [201600/332987 (61%)]	Loss: 0.006530
Train Epoch: 4 [204800/332987 (62%)]	Loss: 0.013759
Train Epoch: 4 [208000/332987 (62%)]	Loss: 0.003928
Train Epoch: 4 [211200/332987 (63%)]	Loss: 0.018783
Train Epoch: 4 [214400/332987 (64%)]	Loss: 0.024181
Train Epoch: 4 [217600/332987 (65%)]	Loss: 0.005473
Train Epoch: 4 [220800/332987 (66%)]	Loss: 0.029957
Train Epoch: 4 [224000/332987 (67%)]	Loss: 0.038422
Train Epoch: 4 [227200/332987 (68%)]	Loss: 0.086862
Train Epoch: 4 [230400/332987 (69%)]	Loss: 0.008583
Train Epoch: 4 [233600/332987 (70%)]	Loss: 0.035062
Train Epoch: 4 [236800/332987 (71%)]	Loss: 0.001783
Train Epoch: 4 [240000/332987 (72%)]	Loss: 0.023192
Train Epoch: 4 [243200/332987 (73%)]	Loss: 0.000731
Train Epoch: 4 [246400/332987 (74%)]	Loss: 0.010067
Train Epoch: 4 [249600/332987 (75%)]	Loss: 0.148109
Train Epoch: 4 [252800/332987 (76%)]	Loss: 0.010359
Train Epoch: 4 [256000/332987 (77%)]	Loss: 0.018811
Train Epoch: 4 [259200/332987 (78%)]	Loss: 0.030441
Train Epoch:

Train Epoch: 6 [44800/332987 (13%)]	Loss: 0.002870
Train Epoch: 6 [48000/332987 (14%)]	Loss: 0.137996
Train Epoch: 6 [51200/332987 (15%)]	Loss: 0.015037
Train Epoch: 6 [54400/332987 (16%)]	Loss: 0.002090
Train Epoch: 6 [57600/332987 (17%)]	Loss: 0.007295
Train Epoch: 6 [60800/332987 (18%)]	Loss: 0.008832
Train Epoch: 6 [64000/332987 (19%)]	Loss: 0.010191
Train Epoch: 6 [67200/332987 (20%)]	Loss: 0.008626
Train Epoch: 6 [70400/332987 (21%)]	Loss: 0.082952
Train Epoch: 6 [73600/332987 (22%)]	Loss: 0.010761
Train Epoch: 6 [76800/332987 (23%)]	Loss: 0.004870
Train Epoch: 6 [80000/332987 (24%)]	Loss: 0.039306
Train Epoch: 6 [83200/332987 (25%)]	Loss: 0.002738
Train Epoch: 6 [86400/332987 (26%)]	Loss: 0.072524
Train Epoch: 6 [89600/332987 (27%)]	Loss: 0.000947
Train Epoch: 6 [92800/332987 (28%)]	Loss: 0.007098
Train Epoch: 6 [96000/332987 (29%)]	Loss: 0.002874
Train Epoch: 6 [99200/332987 (30%)]	Loss: 0.001639
Train Epoch: 6 [102400/332987 (31%)]	Loss: 0.018022
Train Epoch: 6 [105600/332987 

Train Epoch: 7 [220800/332987 (66%)]	Loss: 0.002386
Train Epoch: 7 [224000/332987 (67%)]	Loss: 0.015195
Train Epoch: 7 [227200/332987 (68%)]	Loss: 0.014855
Train Epoch: 7 [230400/332987 (69%)]	Loss: 0.001934
Train Epoch: 7 [233600/332987 (70%)]	Loss: 0.042575
Train Epoch: 7 [236800/332987 (71%)]	Loss: 0.007519
Train Epoch: 7 [240000/332987 (72%)]	Loss: 0.009602
Train Epoch: 7 [243200/332987 (73%)]	Loss: 0.001990
Train Epoch: 7 [246400/332987 (74%)]	Loss: 0.051574
Train Epoch: 7 [249600/332987 (75%)]	Loss: 0.001826
Train Epoch: 7 [252800/332987 (76%)]	Loss: 0.002280
Train Epoch: 7 [256000/332987 (77%)]	Loss: 0.002018
Train Epoch: 7 [259200/332987 (78%)]	Loss: 0.058622
Train Epoch: 7 [262400/332987 (79%)]	Loss: 0.021916
Train Epoch: 7 [265600/332987 (80%)]	Loss: 0.003297
Train Epoch: 7 [268800/332987 (81%)]	Loss: 0.006752
Train Epoch: 7 [272000/332987 (82%)]	Loss: 0.014814
Train Epoch: 7 [275200/332987 (83%)]	Loss: 0.003632
Train Epoch: 7 [278400/332987 (84%)]	Loss: 0.000583
Train Epoch:

Train Epoch: 9 [64000/332987 (19%)]	Loss: 0.000358
Train Epoch: 9 [67200/332987 (20%)]	Loss: 0.013037
Train Epoch: 9 [70400/332987 (21%)]	Loss: 0.003248
Train Epoch: 9 [73600/332987 (22%)]	Loss: 0.000427
Train Epoch: 9 [76800/332987 (23%)]	Loss: 0.003115
Train Epoch: 9 [80000/332987 (24%)]	Loss: 0.014842
Train Epoch: 9 [83200/332987 (25%)]	Loss: 0.001740
Train Epoch: 9 [86400/332987 (26%)]	Loss: 0.001067
Train Epoch: 9 [89600/332987 (27%)]	Loss: 0.005257
Train Epoch: 9 [92800/332987 (28%)]	Loss: 0.003975
Train Epoch: 9 [96000/332987 (29%)]	Loss: 0.057031
Train Epoch: 9 [99200/332987 (30%)]	Loss: 0.006414
Train Epoch: 9 [102400/332987 (31%)]	Loss: 0.003508
Train Epoch: 9 [105600/332987 (32%)]	Loss: 0.000756
Train Epoch: 9 [108800/332987 (33%)]	Loss: 0.000824
Train Epoch: 9 [112000/332987 (34%)]	Loss: 0.006167
Train Epoch: 9 [115200/332987 (35%)]	Loss: 0.001879
Train Epoch: 9 [118400/332987 (36%)]	Loss: 0.000953
Train Epoch: 9 [121600/332987 (37%)]	Loss: 0.028236
Train Epoch: 9 [124800/3

In [14]:
del train_x_np
del train_y_np
#del test_x_np
#del test_y_np
del train_loader
#del test_loader
del train_dataset
#del test_dataset

In [15]:
gc.collect()
torch.cuda.empty_cache()

In [16]:
pred = np.load('test.npy')
pred = np.array([resize(img) for img in tqdm(pred)])
pred = np.array([i[:, :, np.newaxis] for i in tqdm(pred)])
pred_dataset = Hieroglyph_data(pred, train = False)
pred_loader = torch.utils.data.DataLoader(pred_dataset, batch_size = batch_size, shuffle = False)

100%|██████████| 83247/83247 [00:00<00:00, 1372556.30it/s]


In [17]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    with torch.no_grad():
        for i, images in enumerate(data_loader):
            images = Variable(images.view(-1, 1, h, w))

            images = images.to(device)

            outputs = model(images)

            pred = outputs.cpu().data.max(1, keepdim=True)[1]

            test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [18]:
pred_y = prediciton(pred_loader)
pred_y = pred_y.cpu().numpy().flatten()

dct = {v:k for k,v in dct.items()}

pred_y = [dct[code] for code in pred_y]

In [19]:
import pandas as pd
results = pd.Series(pred_y, name="Category", dtype = 'object')

submission = pd.concat([pd.Series(range(1,83248), name = "Id", dtype='object'), results],axis = 1)

submission.to_csv("dpn92_64_64_no_validation.csv",index=False)